# Multi-Modal Data Analysis Workflow

**ASIST Study 3 Dataset**

#### **Team 000315 Pre-Result Analysis**

Engineer exibited poor skill during the hands on training. It took him a very long time to complete the individual training which resulted in the mission timer running out during the team training section. Once the timer expired, the hands-on training trial was restarted and participants again completed the individual training. Engineer's second attempt still took over 5 minutes to complete individual training. The competency test was then started at 13m remaining on the timer, the engineer took a full 5 minutes to complete the competency test. Experiment team decided that engineer should be allowed to proceed to avoid rescheduling dispite the evident lack of minecraft skill.

## Objective

Analyze team performance data across four modalities:

1. JSON behavior logs
2. Video recordings
3. Chat transcripts

Identify correlations between AI interventions and team outcomes.

## Note

All datasets were taken from official CHART ASIST Study 3 Dataset available at ASU official repository.

#### Subset used :

| Team ID | ASI ID        | trial   | intervention_recipent     |
| ------- | ------------- | ------- | ------------------------- |
| 000315  | ASI-CMURI-TA1 | T000829 | E001211, E001215, E001155 |



**AI Agent Action signals** -

1. RemindTransporterBeep
2. InformAboutTriagedVictim
3. RemindMedicToInformAboutTriagedVicti
4. TriageCriticalVictim
5. EvacuateCriticalVictim
6. EncouragePlayerProximityToMedicIHMCDyad
7. RemindChangeMarke
8. RemindRubblePerturbatio
9. EvacuationZoneDistanc
10. TeamSawVictimMarke
11. TimeElapse
12. StartEvacuatio

**Agents location**-

1. Location of the agent in the map -> Room Name

**Agent Action**-

1. Transporting victims
2. Performing their role task including stabilizing victims
3. wakening up critical victims
4. placing marking block-> Regular, A, B, C
5. placing marking block for threat rooms
6. removing rubbles
7. detecting victims

## V1 Dataframe

Timestamp, AI Message, AI Action Class, Transporter Message, Engineer Message, Medic Message is extracted from transcript.csv

We use Multimodal LLM analysis to analyze video data to give states and locations of agents + victims throughout the experiment

States tells us what the agent is doing
Locations tells use where the agent is located in the map

| Time Stamp (Transcript) | Asi Message (Transcript) | Asi Action Class (Transcript) | Transporter Message (Transcript) | Engineer Message (Transcript) | Medic Message (Transcript) | Transporter State (LLM) | Engineer State (LLM) | Medic State (LLM) | Transporter location (LLM) | Engineer location (LLM) | Medic location (LLM) | Victim Location (LLM) |
| ---------- | ----------- | ---------------- | ------------------- | ---------------- | ------------- | ----------------- | -------------- | ----------- | -------------------- | ----------------- | -------------- | --------------- |
| 11:23:01   | N/A         | N/A              | N/A                 | N/A              | N/A           | N/A               | N/A            | N/A         | N/A                  | N/A               | N/A            | N/A             |

## Final Dataframe

States and locations are fused together using LLM analysis to form one action_state column signifying their role in a situation

| timestamp (Transcript) | asi_reason (Transcript) | asi_action (Transcript) | transporter_message (Transcript) | engineer_message (Transcript) | medic_message (Transcript) | transporter_action_state (LLM) | engineer_action_state (LLM) | medic_action_state (LLM) | victim_location (LLM) | 
| --------- | --------- | --------- | ------------------ | --------------- | ------------ | --------------- | --------------- | ------------ | --------------- |
|           |           |           |                    |                 |              |                 |                 |              |                 |


We then utilize another LLM to finally provide ASI Advice score and team score for their actions and LLM's reasoning behiind that

| timestamp (Transcript) | asi_reason (Transcript) | asi_action (Transcript) | transporter_message (Transcript) | engineer_message (Transcript) | medic_message (Transcript) | transporter_action_state (LLM) | engineer_action_state (LLM) | medic_action_state (LLM) | victim_location (LLM) | team_score (LLM) | asi_advice_score (LLM) | team_score_reason (LLM) | asi_advice_score_reason (LLM)
| --------- | --------- | --------- | ------------------ | --------------- | ------------ | --------------- | ---------- | --------------- | --------------- |--------------- |--------------- | --------------- | --------------- |
|     22:03      |   You guys should do [asi_action_class] because...        | 1. RemindTransporterBeep <br/>2. InformAboutTriagedVictim <br/>3. RemindMedicToInformAboutTriagedVicti <br/>4. TriageCriticalVictim <br/>5. EvacuateCriticalVictim <br/>6.EncouragePlayerProximityToMedicIHMCDyad <br/>7. RemindChangeMarke <br/>8. RemindRubblePerturbatio <br/>9. EvacuationZoneDistanc <br/>10. TeamSawVictimMarke <br/>11. TimeElapse <br/>12. StartEvacuatio|     I'm coming for you medic               |     This is more important               |     I can't help you!               |     Carrying a victim from b4 to g4 room               |     Clearing rubbles in threat room for medic  at a9 room          |     waking up critical victim at g5 room        |    next to medic, far from engineer, close to transporter            |  40%          |     75%           | team was inconsistent with their tasks, especially... | asi's advice is particularly useful because... |


## AI Instruct Modal

We finetune pretrained LLM on our data for understanding minecraft test bed for asist thoroughly and deeply. Then we utilize it to perform-

1. **Multimodal DataAnalysis of Video Data** 

To give information about agent locations and their actions

2. **Text Fusion Data Analysis**

To fuse meanings and relationships between agent communication, their location and state in given situations to a single column

3. **Scoring Analysis**

To score humans and ASI's advice on team work communication and collaboration data   



# Installing dependencies

In [ ]:
import cv2
from pathlib import Path
import json
import pandas as pd
# import openai
import matplotlib.pyplot as plt
import os
import warnings
# import torch
warnings.filterwarnings("ignore")
import json

# Preparing Dataset

### 1. JSON Logs Processing
#### Objective
Extract structured data from nested JSON logs containing:
- Team actions
- AI intervention timestamps
- Mission outcomes


In [ ]:


# def parse_json_logs(input_path: Path, output_path: Path) -> pd.DataFrame:
#     """Flatten nested JSON logs into structured format"""
#     with open(input_path, 'r') as f:
#         data = [json.loads(line) for line in f]
    
#     df = pd.json_normalize(data, sep='_')
#     df.to_csv(output_path, index=False)
#     return df

# # Process all trial messages
# input_files = [
#     Path("data/json_logs/HSRData_TrialMessages_Trial-T000603_..."),
#     Path("data/json_logs/HSRData_TrialMessages_Trial-T000639_..."),
#     Path("data/json_logs/HSRData_TrialMessages_Trial-T000671_...")
# ]

# output_dir = Path("data/processed/json_parsed/")
# output_dir.mkdir(parents=True, exist_ok=True)

# for file in input_files:
#     output_file = output_dir / f"{file.stem}_parsed.csv"
#     df = parse_json_logs(file, output_file)
#     print(f"Processed {len(df)} records from {file.name}")



# Preparing Finetuned Model

### Objective 

Finetune the pretrained instruct model on testbed refined high quality data and information about agents, environment, locations. 

[ Maybe : Train model to understand video data as well ]

### Engineer Instruct POV Model

Finetune the pretrained instruct model on testbed refined high quality data and information about agents actions in minecraft video. 

[ Maybe : Train model to understand video data as well ]

### Medic Instruct POV 

Finetune the pretrained instruct model on testbed refined high quality data and information about agents actions in minecraft video. 

[ Maybe : Train model to understand video data as well ]

### Transporter Instruct POV Model


Finetune the pretrained instruct model on testbed refined high quality data and information about agents actions in minecraft video. 

[ Maybe : Train model to understand video data as well ]

### Experimenter's Map Instruct Model


Finetune the pretrained instruct model on testbed refined high quality data and information about victim locations and agent locations in live map. 

[ Maybe : Train model to understand video data as well ]

### Player's Action State Fusor Model


Fuses player actions and their location in maps to form a rich action state data. 


### Scoring Model

Consideres all important data and gives score to each intervention's team and asi advice with reasoning.

# Data Analysis


## Transcript.csv analysis

In [2]:
df = pd.read_csv("/mnt/c/Users/Som/Desktop/CHART ASIST/Study3_Analysis/data/transcripts/transcript.csv")
df = df[1:]

NameError: name 'pd' is not defined

In [ ]:
df.head(5)

#### Cleaning Data

Originally it had many columns and we reduced it down to only the ones which had data 



In [ ]:
df["asi_message"] = df["intervention_message"].replace("{}", "")
df["team_message"] = df["speech_message"].replace("{}", "")

Adding a intervention_class column from explanation string

In [ ]:
# Extract intervention_class from explanation strings
df['intervention_class'] = df['explanation'].str.extract(
    r"'intervention_class'\s*:\s*'([^']*)'"
)

# Create binary columns for each unique intervention class
intervention_classes = df['intervention_class'].dropna().unique()
for cls in intervention_classes:
    df[cls] = df['intervention_class'].eq(cls).astype(int)

# Cleanup intermediate column
df = df.drop(columns=['intervention_class'])

# Clean column names by removing 'Intervention' suffix
df = df.rename(columns=lambda col: col[:-13] if col.endswith('Intervention') else col)



We don't need unique team ids, asi ids, date, trial id, intervention_recipent id or explanation since we have extracted the unique class

In [ ]:

df = df.drop(columns=["team","asi","date","intervention_recipent","intervention_message","speech_message","trial"], axis=1)



In [ ]:
df.sample(15)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(inplace=True)

 Drop duplicated values

In [ ]:
df = df.drop_duplicates()

## Experimenter's Map LLM Data Analysis - Multimodal

### Objective -

Analyzing gameplay videos with AI, to output -

**1. Agents location**-

1. Location of the agent in the map -> Room Name

**2. Victims location**-

### Prompt for AI Modal 

You are 

### Response Type

```json
{
    agent: "engineer | transporter | medic | victim", location: "roomb12|roomh12..."
}
```


### Divide the video in 25ms parts

In [ ]:
## TODO : Use ffmpeg to divide videos into frames

### Pass video chunks to LLM

In [ ]:
## call modal apis and retreive json

### Convert Json to dataframe

In [ ]:
## Convert the json to csv column -> States and Locations

## Player's POV LLM Data Analysis - Multimodal

### Objective -

Analyzing gameplay videos with AI, to output -

**1. Agent actions**

### Prompt for AI Modal 

You are 

### Response Type

```json
{
    agent: "engineer | transporter | medic | victim", action: "chat|door|ItemDrop|ItemEquipped|ItemPickup|ItemUsed|Lever|PlayerJumped|PlayerSprinting|Triage|ProximityBlockInteraction|ToolUsed|CompetencyTask
|ToolDepleted|MarkerPlaced|MarkerRemoved|RubbleCollapse|VictimEvacuated|VictimPickedUp|VictimPlaced|RubbleDestroyed|PerturbationRubbleLocations
|VictimsExpired"
}
```


### Divide the video in 25ms parts

In [ ]:
## TODO : Use ffmpeg to divide videos into frames

### Pass video chunks to LLM

In [ ]:
## call modal apis and retreive json

### Convert Json to dataframe

In [ ]:
## Convert the json to csv column -> States and Locations

## Fusion LLM Text Analysis



### Objective

Use LLM to analyze each agent's communication, location and state to form one [agent]_action_state 

**1. Map State**-

Location of the agent in the map -> Room Name

**2. Agent Action**-

1. chat
2. door
3. ItemDrop
4. ItemEquipped
5. ItemPickup
6. ItemUsed
7. Lever
8. PlayerJumped
9. PlayerSprinting
10. Scoreboard
11. Triage
12. RoleSelected
13. ProximityBlockInteraction
14. PlayerFrozenStateChange
15. ToolUsed
16. CompetencyTask
17. TrainingTask
18. ToolDepleted
19. MarkerPlaced
20. MarkerRemoved
21. RubbleCollapse
22. VictimEvacuated
23. VictimPickedUp
24. VictimPlaced
25. RubbleDestroyed
26. Signal
27. Pause
28. MissionState
29. Perturbation
30. PlanningStage
31. PerturbationRubbleLocations
32. location
33. proximity
34. dyad
35. VictimsExpired
36. PuzzleTextSummary
37. dialogue_event
38. TrialState

**3. Agent Message**-

Raw message extracted from transcript.csv

### Prompt for AI Modal 

You are 

### Response Type

```json
{
    agent: "engineer | transporter | medic | victim", [agent]_action_state " [Agent] was likely doing this at this location " 
}
```

### Pass each row to LLM and retreive JSON

In [ ]:
# TODO 

### Convert JSON to a single dataframe of csv -> [agent]_action_state and append it to csv

In [ ]:
# TODO

## Score LLM Text Analysis



### Objective

Use LLM to analyze agent's advice and team's communication and collaboration to form team_score and asi_advice score

| timestamp | asi_reason | asi_action | transporter_message | engineer_message | medic_message | transporter_action_state | engineer_action_state | medic_action_state | victim_location | 
| --------- | --------- | --------- | ------------------ | --------------- | ------------ | --------------- | --------------- | ------------ | --------------- |
|           |           |           |                    |                 |              |                 |                 |              |                 |

### Prompt for AI Modal 

You are 

### Response Type

```json
{
    asi_advice_score: "number %", team_score : "number %" 
}
```

### Pass each row to LLM and retreive JSON

In [ ]:
# TODO 

### Convert JSON to a single dataframe of csv -> team_score and asi_advice score and append it to csv

In [ ]:
# TODO